In [ ]:
!wget https://raw.githubusercontent.com/AbhiDhariwal/Hindi-NER/master/CoNLL2Spacy.py

--2023-05-04 04:25:40--  https://raw.githubusercontent.com/AbhiDhariwal/Hindi-NER/master/CoNLL2Spacy.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1128 (1.1K) [text/plain]
Saving to: ‘CoNLL2Spacy.py’

CoNLL2Spacy.py      100%[===================>]   1.10K  --.-KB/s    in 0s      

2023-05-04 04:25:40 (76.5 MB/s) - ‘CoNLL2Spacy.py’ saved [1128/1128]



In [ ]:
from CoNLL2Spacy import *

In [ ]:
file = open("/content/nervalweather.txt", "r",encoding = "utf-8") 
valList = []
for line in file:
    valList.append(line[:-1])
valList[:15]

['आज U-date',
 'मौसम O',
 'कैसा O',
 'है O',
 '',
 'क्या O',
 'हरिद्वार U-location',
 'में O',
 'बहुत O',
 'ठंड U-weather_type',
 'है O',
 'अभी U-date',
 '',
 'आज U-date',
 'इतनी O']

In [ ]:
TEST_DATA = conll2spacy(valList)

In [ ]:
file = open("/content/nervalweather.txt", "r",encoding = "utf-8") 
trainList = []
for line in file:
    trainList.append(line[:-1])
trainList[:15]

['आज U-date',
 'मौसम O',
 'कैसा O',
 'है O',
 '',
 'क्या O',
 'हरिद्वार U-location',
 'में O',
 'बहुत O',
 'ठंड U-weather_type',
 'है O',
 'अभी U-date',
 '',
 'आज U-date',
 'इतनी O']

In [ ]:
TRAIN_DATA = conll2spacy(trainList)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec

--2023-05-04 04:29:10--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.hi.vec
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.249.39.89, 13.249.39.25, 13.249.39.6, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.249.39.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 416644179 (397M) [binary/octet-stream]
Saving to: ‘wiki.hi.vec’

wiki.hi.vec         100%[===================>] 397.34M  30.7MB/s    in 12s     

2023-05-04 04:29:23 (31.8 MB/s) - ‘wiki.hi.vec’ saved [416644179/416644179]



In [ ]:
import spacy
import numpy
import pickle

In [ ]:
lang = "hi"
vectors_loc = "wiki.hi.vec"
nlp = spacy.blank(lang)    
with open(vectors_loc, "rb") as file_:        
    header = file_.readline()        
    nr_row, nr_dim = header.split()        
    nlp.vocab.reset_vectors(width=int(nr_dim))        
    for line in file_:            
        line = line.rstrip().decode("utf8")            
        pieces = line.rsplit(" ", int(nr_dim))            
        word = pieces[0]            
        vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
        nlp.vocab.set_vector(word, vector)  # add the vectors to the vocab   

In [ ]:
text = "भारी बारिश के कारण आज कार्यालय बंद रहेगा"    
doc = nlp(text)    
print("similarity btw",doc[0] , "and", doc[3]," :-", doc[0].similarity(doc[3]))

similarity btw भारी and कारण  :- 0.4151364266872406


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.0.1
    Uninstalling pip-23.0.1:
      Successfully uninstalled pip-23.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install --upgrade spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
for ent in doc.ents:
     print(ent.text,"|",ent.label_,"|",spacy.explain(ent.label_))

In [ ]:
from spacy import displacy
displacy.render(doc,style="ent")

'<div class="entities" style="line-height: 2.5; direction: ltr">भारी बारिश के कारण आज कार्यालय बंद रहेगा</div>'

In [ ]:
import spacy
import random 

from spacy.util import minibatch, compounding

from spacy.scorer import Scorer

In [ ]:
def train_spacy(TRAIN_DATA,TEST_DATA,iterations,droprate = 0.5,modelName = "modelTrained"):

    # loading hindi model and using vector from fasttext
    lang = "hi"
    vectors_loc = "wiki.hi.vec"
    modiner = spacy.blank(lang)    
    with open(vectors_loc, "rb") as file_:        
        header = file_.readline()        
        nr_row, nr_dim = header.split()        
        modiner.vocab.reset_vectors(width=int(nr_dim))        
        for line in file_:            
            line = line.rstrip().decode("utf8")            
            pieces = line.rsplit(" ", int(nr_dim))            
            word = pieces[0]            
            vector = numpy.asarray([float(v) for v in pieces[1:]], dtype="f")            
            modiner.vocab.set_vector(word, vector)  # add the vectors to the vocab    

#     modiner = spacy.blank('en')  # create blank Language class
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in modiner.pipe_names:
        ner = modiner.create_pipe('ner')
        modiner.add_pipe(ner, last=True)
     
    # setting up f1score
    f1score = 0.0000

    
    # add labels that will be involved in training 
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    other_pipes = [pipe for pipe in modiner.pipe_names if pipe != 'ner']
    with modiner.disable_pipes(*other_pipes):  # only train NER
        optimizer = modiner.begin_training()
        
        # --Iterations Starts--
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            #--Shuffling Traning Data--
            random.shuffle(TRAIN_DATA)
            losses = {}
            
                      
                    
            # batch Traning For better Training and Learning of model
            batches = minibatch(TRAIN_DATA, size=compounding(2.0, 16.0, 1.01))
            for batch in batches:
                texts, annotations = zip(*batch)
                modiner.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=droprate,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print(losses)
            
            
            # Evaluating the Current Model Score on test data
            results = evaluate(modiner, TEST_DATA)
            print("Current Score :-",results["ents_f"], "Precision  :-",results["ents_p"], "Recall  :-",results["ents_r"])
            

            
            # loading previous best saved model in start of traning 
            if f1score == 0.00:
                try:                    
                    pnlp = spacy.load(modelName)
                    result = evaluate(pnlp, TEST_DATA) # calling evaluate function 
                    f1score = result["ents_f"]
                except:
                    print("Previous Model not found")
                    
            print("Best Sccore :- ",f1score)
            print("------------------------------------")
            # finding out the best score
            if f1score < results["ents_f"]:
                f1score = results["ents_f"]
                
                # Save our trained Model if the score if grater than best score else no change in previous model
                modiner.to_disk(modelName)
                
    print("-----Best Model is Saved-----")

In [ ]:
def evaluate(ner_model, examples):
    scorer = Scorer()
     
    #loading tags for each input and Evaluating them
    for input_, annotations in examples:
        tags = []
        # loading text
        doc_gold_text = ner_model.make_doc(input_)
        
        #loading all tags for that text
        for ent in annotations.get('entities'):
            tags.append(ent)
            
        # Evaluating the tags    
        gold = GoldParse(doc_gold_text, entities=tags)
        pred_value = ner_model(input_)
        scorer.score(pred_value, gold)
        
        
    return scorer.scores

In [ ]:
def loadNERModel(modelName = "modelTrained"):
    nlp = spacy.load(modelName)
    return nlp

In [ ]:
def score(model,TEST_DATA):
    result = evaluate(model, TEST_DATA) # calling evaluate function 
    f1score = result["ents_f"]
    precision = result["ents_p"]
    recall = result["ents_r"]
    print("F1 score of Model is :-",f1score)
    print("Precision of Model is :-",precision)
    print("Recall of Model is :-",recall)

In [ ]:
from spacy.language import Language
from spacy.pipeline import EntityRecognizer

@Language.component('custom_ner')
def custom_ner_component(doc):
    # Your custom NER logic here
    return doc

# Add the component to the pipeline
nlp.add_pipe('custom_ner')

<function __main__.custom_ner_component(doc)>

In [ ]:
def train_spacy(TRAIN_DATA, TEST_DATA, iterations, droprate=0.5, modelName="hindiNER"):
    nlp = spacy.blank('hi')
    print("Created blank Hindi model")
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe('ner', last=True)
        print("Added NER component to pipeline")
    else:
        ner = nlp.get_pipe('ner')
        
    # Adding labels to the ner
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
    
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    
    with nlp.disable_pipes(*other_pipes):
        optimizer = nlp.begin_training()
        dropout = droprate
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(texts, annotations, sgd=optimizer, drop=dropout, losses=losses)
            print("Losses at iteration ", itn, " - ", losses)
            
    # Testing the model
    for text, _ in TEST_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        
    # Save the model
    output_dir = Path('contentnervalweather.txt')
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir / modelName)
    print("Model saved to ", output_dir / modelName)